### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
## Laboratory 1 Solution
### Cooperating with: Luca Lodesani (s346978)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

In [16]:
import numpy as np
import math

In [110]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [120]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [122]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

## Base Hill Climbing 

In [117]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = sol.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

# --- Hill climbing loop ---
iterations = 10000
for _ in range(iterations):

    new_solution = tweak(solution)

    # generate new solution if invalid
    if not is_valid(new_solution):
        continue
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")


Total value: 909
Zaino 1 value: 485
Zaino 2 value: 157
Zaino 3 value: 267
Zaino 1 weights: [349 479], constraints: [614 496]
Zaino 2 weights: [202 156], constraints: [244 644]
Zaino 3 weights: [204 214], constraints: [273 216]


## Tabu Search Hill Climbing

In [118]:
NUM_SAMPLES = 10  # number of neighbors to sample

In [ ]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = sol.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i] 
    return new_solution

# --- Hill climbing loop ---
iterations = 10000
tabu_list = set()
for _ in range(iterations):

    possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [sol for sol in possible_solutions if is_valid(sol) and sol.flatten().tobytes() not in tabu_list]

    # if no valid candidates, continue
    if not len(candidates):
        continue

    new_solution = max(candidates, key=value)
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        tabu_list.add(solution.flatten().tobytes())
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")
    


## Simulated Annealing Hill Climbing

In [123]:
# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

# more versatile tweak 
def tweak(sol):
    """
    Remove random item from all knapsacks and possibly add it to a random knapsack
    """
    new_solution = sol.copy()
    item_to_tweak = rng.integers(0, NUM_ITEMS)
    destination_knapsack = rng.integers(0, NUM_KNAPSACKS + 1)
    
    new_solution[:, item_to_tweak] = False
    
    if destination_knapsack < NUM_KNAPSACKS:
        new_solution[destination_knapsack, item_to_tweak] = True
        
    return new_solution
    
# Simulated Annealing Loop
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)
current_fitness = value(solution)

# SA parameters
initial_temp = 100.0
final_temp = 0.1
alpha = 0.999    # Annealing schedule rate
temp = initial_temp
iterations = 10000

for i in range(iterations):
    new_solution = tweak(solution)
    new_fitness = value(new_solution)
    
    if not is_valid(new_solution):
        continue
    
    # Calculate the change in fitness
    delta = new_fitness - current_fitness
    
    # If the new solution is better, always accept it
    if delta > 0:
        solution = new_solution
        current_fitness = new_fitness
    # If the new solution is worse, accept it with a probability
    else:
        acceptance_probability = math.exp(delta / temp)
        if rng.random() < acceptance_probability:
            solution = new_solution
            current_fitness = new_fitness

    # Decrease the temperature
    temp = temp * alpha
    if temp < final_temp:
        temp = final_temp
        

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")
    

Total value: 1143541
Zaino 1 value: 13923
Zaino 2 value: 11961
Zaino 3 value: 11359
Zaino 4 value: 9798
Zaino 5 value: 15213
Zaino 6 value: 9547
Zaino 7 value: 11567
Zaino 8 value: 12938
Zaino 9 value: 9710
Zaino 10 value: 8718
Zaino 11 value: 13908
Zaino 12 value: 12738
Zaino 13 value: 12122
Zaino 14 value: 11045
Zaino 15 value: 10832
Zaino 16 value: 12255
Zaino 17 value: 10268
Zaino 18 value: 12612
Zaino 19 value: 9700
Zaino 20 value: 10213
Zaino 21 value: 10878
Zaino 22 value: 13488
Zaino 23 value: 11295
Zaino 24 value: 12350
Zaino 25 value: 13538
Zaino 26 value: 11996
Zaino 27 value: 11605
Zaino 28 value: 9500
Zaino 29 value: 13414
Zaino 30 value: 13870
Zaino 31 value: 7321
Zaino 32 value: 10500
Zaino 33 value: 12040
Zaino 34 value: 12143
Zaino 35 value: 13837
Zaino 36 value: 11371
Zaino 37 value: 9633
Zaino 38 value: 12095
Zaino 39 value: 12118
Zaino 40 value: 10257
Zaino 41 value: 9852
Zaino 42 value: 12964
Zaino 43 value: 11121
Zaino 44 value: 9855
Zaino 45 value: 9923
Zaino 46 